# For figure 6.25

Temperature response to forcing 2020-2100

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as pl
import os
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
import matplotlib.patches as mp
from netCDF4 import Dataset
import warnings

from ar6.utils.h5 import *

In [2]:
# TODO - sort out figure sizing

pl.rcParams['figure.figsize'] = (18/2.54, 11/2.54)
pl.rcParams['font.size'] = 11
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['ytick.major.size'] = 0
pl.rcParams['xtick.top'] = True
pl.rcParams['figure.dpi'] = 150

In [3]:
results = load_dict_from_hdf5('../data_output_large/twolayer_SSPs.h5')

In [4]:
results.keys()

dict_keys(['ssp119', 'ssp119_climuncert', 'ssp119_remove_aerosol', 'ssp119_remove_aerosol_climuncert', 'ssp119_remove_bc_on_snow', 'ssp119_remove_bc_on_snow_climuncert', 'ssp119_remove_ch4', 'ssp119_remove_ch4_climuncert', 'ssp119_remove_co2', 'ssp119_remove_co2_climuncert', 'ssp119_remove_hfc', 'ssp119_remove_hfc_climuncert', 'ssp119_remove_o3', 'ssp119_remove_o3_climuncert', 'ssp119_remove_other_anthro', 'ssp119_remove_other_anthro_climuncert', 'ssp119_remove_other_ghg', 'ssp119_remove_other_ghg_climuncert', 'ssp119_remove_slcf', 'ssp119_remove_slcf_climuncert', 'ssp126', 'ssp126_climuncert', 'ssp126_remove_aerosol', 'ssp126_remove_aerosol_climuncert', 'ssp126_remove_bc_on_snow', 'ssp126_remove_bc_on_snow_climuncert', 'ssp126_remove_ch4', 'ssp126_remove_ch4_climuncert', 'ssp126_remove_co2', 'ssp126_remove_co2_climuncert', 'ssp126_remove_hfc', 'ssp126_remove_hfc_climuncert', 'ssp126_remove_o3', 'ssp126_remove_o3_climuncert', 'ssp126_remove_other_anthro', 'ssp126_remove_other_anthro_cl

In [5]:
results['ssp245']['surface_temperature'].shape

(351, 2049)

In [6]:
results['ssp245']['surface_temperature'][269].mean()

1.2853468928398741

In [7]:
forcings = ['ch4', 'hfc', 'o3', 'aerosol', 'bc_on_snow', 'slcf']
scenarios = ['ssp119','ssp126','ssp245','ssp370','ssp370-lowNTCF-aerchemmip','ssp370-lowNTCF-gidden','ssp434','ssp460','ssp534-over','ssp585']
for scenario in scenarios:
    for forcing in forcings:
        print(scenario, forcing, (results[scenario]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351]).mean())

ssp119 ch4 0.147529067352141
ssp119 hfc 0.01275680288532077
ssp119 o3 0.10542350956390295
ssp119 aerosol -0.15915953218087653
ssp119 bc_on_snow 0.00693205643504798
ssp119 slcf 0.11348190405554855
ssp126 ch4 0.15648961988754184
ssp126 hfc 0.015416873431177924
ssp126 o3 0.11029918478440984
ssp126 aerosol -0.1607839075264361
ssp126 bc_on_snow 0.010240107907998465
ssp126 slcf 0.13166187848440436
ssp245 ch4 0.2832096819082067
ssp245 hfc 0.06333574552671715
ssp245 o3 0.20359174343413988
ssp245 aerosol -0.27277285742001356
ssp245 bc_on_snow 0.01774834240578654
ssp245 slcf 0.29511265585448704
ssp370 ch4 0.4944363914703334
ssp370 hfc 0.09091076221769345
ssp370 o3 0.4240110975180323
ssp370 aerosol -0.5646019685123352
ssp370 bc_on_snow 0.06696538861366341
ssp370 slcf 0.5117216713075804
ssp370-lowNTCF-aerchemmip ch4 0.4944363914704428
ssp370-lowNTCF-aerchemmip hfc 0.09091076221768886
ssp370-lowNTCF-aerchemmip o3 0.3543187309774875
ssp370-lowNTCF-aerchemmip aerosol -0.28055992930569745
ssp370-lowNT

In [8]:
(results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][0]).shape

(20, 2049)

In [9]:
forcings = ['ch4', 'hfc', 'o3', 'aerosol', 'bc_on_snow', 'slcf']

AR6_forc = {}
AR6_ecsforc = {}

base_periods = {
#    '1750': 0,
#    '1850-1900': slice(100,151),
#    '1995-2014': slice(245,265),
#    '2010-2019': slice(260,270),
    '2020'     : slice(270,271)
}

for scenario in scenarios:
    AR6_forc[scenario] = {}
    AR6_ecsforc[scenario] = {}
    for forcing in forcings:
        AR6_forc[scenario][forcing] = {}
        AR6_ecsforc[scenario][forcing] = {}
        for base_period in base_periods:
            AR6_forc[scenario][forcing][base_period] = np.zeros((81,5))
            AR6_ecsforc[scenario][forcing][base_period] = np.zeros((81,5))
            for year in range(270,351):
                AR6_forc[scenario][forcing][base_period][year-270,:] = np.percentile(
                    (
                        (results[scenario]['surface_temperature'][year]- results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                        (results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][year] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                    ), (5,16,50,84,95)
                )
                AR6_ecsforc[scenario][forcing][base_period][year-270,:] = np.percentile(
                    (
                        (results['%s_climuncert' % scenario]['surface_temperature'][year] - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                        (results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][year] - results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                    ), (5,16,50,84,95)
                )
    AR6_forc[scenario]['anthro'] = {}
    AR6_ecsforc[scenario]['anthro'] = {}
    for base_period in base_periods:
        AR6_forc[scenario]['anthro'][base_period] = np.zeros((81,5))
        AR6_ecsforc[scenario]['anthro'][base_period] = np.zeros((81,5))
        for year in range(270,351):
            AR6_forc[scenario]['anthro'][base_period][year-270,:] = np.percentile(results[scenario]['surface_temperature'][year] - results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))
            AR6_ecsforc[scenario]['anthro'][base_period][year-270,:] = np.percentile(results['%s_climuncert' % scenario]['surface_temperature'][year] - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))

In [10]:
print(AR6_ecsforc['ssp245']['anthro']['2020'])

[[0.         0.         0.         0.         0.        ]
 [0.02072729 0.02358875 0.02908409 0.03533772 0.03985527]
 [0.04100639 0.04631187 0.05694102 0.06943534 0.07848158]
 [0.06107431 0.06855862 0.08411966 0.10259154 0.115779  ]
 [0.08047137 0.09006722 0.11045495 0.13495889 0.15261002]
 [0.09924359 0.1115619  0.13617375 0.16704996 0.18808397]
 [0.11766671 0.13252786 0.16196938 0.1982289  0.2233076 ]
 [0.13572329 0.15251338 0.18705482 0.22915029 0.2581913 ]
 [0.15322712 0.17227378 0.2117643  0.25971151 0.29354525]
 [0.17056728 0.19212839 0.23649761 0.29045444 0.3289449 ]
 [0.1881224  0.21192663 0.2610404  0.32118035 0.36321456]
 [0.20592068 0.23182272 0.28553057 0.35138459 0.3980978 ]
 [0.22401122 0.2521237  0.31033777 0.3821391  0.43410762]
 [0.24207236 0.27255089 0.33579297 0.41483615 0.47063489]
 [0.26009033 0.29305794 0.3613953  0.4460013  0.50756837]
 [0.27792355 0.31318933 0.38658053 0.47726916 0.54288197]
 [0.29563603 0.33290015 0.41177851 0.50861712 0.58051167]
 [0.3134591  0

In [11]:
for scenario in scenarios:
    print(scenario, AR6_ecsforc[scenario]['anthro']['2020'][:,[0,2,4]])

ssp119 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.00300541e-02  3.04237035e-02  4.40116008e-02]
 [ 3.89822758e-02  6.03381953e-02  8.92860454e-02]
 [ 5.62761333e-02  8.96453958e-02  1.34398487e-01]
 [ 7.24470303e-02  1.17777489e-01  1.79304450e-01]
 [ 8.70496369e-02  1.44219686e-01  2.24581125e-01]
 [ 9.96665630e-02  1.69677052e-01  2.68462515e-01]
 [ 1.11081797e-01  1.93568410e-01  3.12012371e-01]
 [ 1.20971703e-01  2.16020530e-01  3.52910779e-01]
 [ 1.29182244e-01  2.36329306e-01  3.92833624e-01]
 [ 1.36367116e-01  2.55818134e-01  4.32419851e-01]
 [ 1.41677535e-01  2.71818270e-01  4.67644716e-01]
 [ 1.45701776e-01  2.84929805e-01  4.96006628e-01]
 [ 1.47637390e-01  2.94925396e-01  5.20717513e-01]
 [ 1.48344109e-01  3.02604586e-01  5.41196521e-01]
 [ 1.48822338e-01  3.08766859e-01  5.58682604e-01]
 [ 1.48628691e-01  3.12438941e-01  5.73685215e-01]
 [ 1.47649706e-01  3.15630901e-01  5.86086025e-01]
 [ 1.46406801e-01  3.18048262e-01  5.96117797e-01]
 [ 1.43350991e-01  3.192

In [12]:
output = []
for scenario in scenarios:
    for forcing in ['ch4', 'hfc', 'o3', 'aerosol', 'bc_on_snow', 'slcf', 'anthro']:
        for year in range(270, 351):
            output.append([scenario, forcing, '2020', year+1750,
               AR6_ecsforc[scenario][forcing]['2020'][year-270,0],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,1],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,2],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,3],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,4],
            ])

df = pd.DataFrame(output, columns=['scenario', 'forcing', 'base_period', 'year', 'p05','p16','p50','p84','p95'])
df.to_csv('../data_output/slcf_warming_ranges.csv')
df

,scenario,forcing,base_period,year,p05,p16,p50,p84,p95
0,ssp119,ch4,2020,2020,0.000000,0.000000,0.000000,0.000000,0.000000
1,ssp119,ch4,2020,2021,0.001269,0.001477,0.001835,0.002353,0.002712
2,ssp119,ch4,2020,2022,0.002247,0.002632,0.003350,0.004357,0.005033
3,ssp119,ch4,2020,2023,0.002930,0.003454,0.004515,0.005940,0.006948
4,ssp119,ch4,2020,2024,0.003269,0.003947,0.005302,0.007098,0.008454
...,...,...,...,...,...,...,...,...,...
5665,ssp585,anthro,2020,2096,2.476599,2.774079,3.404203,4.182756,4.794687
5666,ssp585,anthro,2020,2097,2.508499,2.808320,3.448744,4.239259,4.858959
5667,ssp585,anthro,2020,2098,2.540093,2.843322,3.493960,4.295741,4.923941
5668,ssp585,anthro,2020,2099,2.571864,2.879303,3.539894,4.351773,4.988523
